In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
from IPython.display import display

from scholar_scraper import *

In [2]:
# get you Scholar id from your page
# (e.g., https://scholar.google.com/citations?user=73-D2jgAAAAJ)
author_id = "73-D2jgAAAAJ"

# register on https://serpapi.com/ and get your SerpApi key
os.environ["SERP_API_KEY"] = "<YOUR_KEY>"

# cache your results
out_path = Path("results")

# depending on the number of your citations, scraping might:
# 1. require a paid serpapi plan
# 2. take a while (~ 7 sec/citation)...
results = scrape_author(author_id, out_path=out_path)

Articles:   0%|          | 0/22 [00:00<?, ?it/s]

In [3]:
# load results as pandas dataframe
author_path = out_path / author_id
results = load_yaml(author_path / "results.yaml")
df = get_citation_df(results, keep_warnings=True)
df.sort_values("article").head(3)

,name,affiliations,email,website,email_domain,warnings,affil_country,affil_name,citations,abbrv_name,article,article_id,citation,citation_id,author_id,affil_country_name
158,Simon Welker,Universität Hamburg,Verified email at unihamburg.de,NaN,unihamburg.de,[],DE,Universität Hamburg,287.0,S Welker,An optimal control perspective on diffusion-ba...,73-D2jgAAAAJ:W7OEmFMy1HYC,DriftRec: Adapting diffusion models to blind J...,None,If3qsuwAAAAJ,Germany
111,Yoshua Bengio,"Professor of computer science, University of M...",Verified email at umontreal.ca,https://yoshuabengio.org/,umontreal.ca,[],CA,Université de Montréal,799012.0,Y Bengio,An optimal control perspective on diffusion-ba...,73-D2jgAAAAJ:W7OEmFMy1HYC,On diffusion models for amortized inference: B...,15947547171699022423,kukA0LcAAAAJ,Canada
113,Maxence Noble,"Ph.D. Student, Ecole Polytechnique, France",Verified email at polytechnique.edu,https://maxencenoble.github.io/,polytechnique.edu,[],FR,Ecole Polytechnique,81.0,M Noble,An optimal control perspective on diffusion-ba...,73-D2jgAAAAJ:W7OEmFMy1HYC,Stochastic Localization via Iterative Posterio...,17344046561291018738,4eGHx3gAAAAJ,France


In [4]:
# authors that most often cited your articles
cols = ["name", "cited_by_name", "affil_name", "website"]
author_df = drop_and_count_duplicates(
    df, ["name", "author_id"], col_name="cited_by_name"
)
author_df.loc[:, cols].sort_values("cited_by_name", ascending=False).head(5)

,name,cited_by_name,affil_name,website
1858,philipp grohs,37.0,Universität Vienna,http://mds.univie.ac.at/
2099,Arnulf Jentzen,31.0,Westfälische Wilhelms-Universität Münster,http://www.ajentzen.de/
57,Julius Berner,25.0,California Institute of Technology,https://jberner.info/
1877,Gitta Kutyniok,20.0,Ludwig-Maximilians-Universität München,http://www.math.lmu.de/~kutyniok
56,Lorenz Richter,20.0,Zuse Institute Berlin,NaN


In [5]:
# citing authors with most citations
cols = ["name", "citations", "affil_name", "website"]
author_df.loc[:, cols].sort_values("citations", ascending=False).head(10)

,name,citations,affil_name,website
63,Yoshua Bengio,798537.0,Université de Montréal,https://yoshuabengio.org/
68,Kevin Black,263059.0,University of Wisconsin - Madison,https://www.physics.wisc.edu/directory/black-k...
62,Aaron Courville,248850.0,Université de Montréal,http://aaroncourville.wordpress.com/
1751,Philip S. Yu,198809.0,University of Illinois at Chicago,http://www.cs.uic.edu/PSYu
223,Sepp Hochreiter,155041.0,Johannes Kepler Universität Linz,https://www.jku.at/en/institute-for-machine-le...
2076,Dacheng Tao,124900.0,Nanyang Technological University,NaN
320,Joshua B. Tenenbaum,110169.0,Massachusetts Institute of Technology,http://web.mit.edu/cocosci/josh.html
1549,Seyedali Mirjalili,109722.0,Griffith University,https://seyedalimirjalili.com/
1300,George Em Karniadakis,106354.0,Brown University,https://www.brown.edu/research/projects/crunch...
578,Michael Wooldridge,85143.0,University of Oxford,http://www.cs.ox.ac.uk/people/michael.wooldridge/


In [6]:
# map of citing authors
get_map(author_df, out_path=author_path, radius_scale=5, radius_log_base=2)

In [7]:
# affiliations that most often cited your articles
cols = ["affil_name", "cited_by_affil", "affil_country_name"]
affil_df = drop_and_count_duplicates(df, ["affil_name"], col_name="cited_by_affil")
affil_df.loc[:, cols].sort_values("cited_by_affil", ascending=False).head(5)

,affil_name,cited_by_affil,affil_country_name
289,Universität Vienna,66.0,Austria
19,"Swiss Federal Institute of Technology, Zurich",56.0,Switzerland
3,California Institute of Technology,46.0,United States
1131,Westfälische Wilhelms-Universität Münster,40.0,Germany
311,Microsoft,38.0,United States


In [8]:
# affiliations citing specific articles
cols = ["citations", "affil_name", "affil_country_name", "cited_by_affil"]
for article in df.loc[:, "article"].unique():
    print(article)
    article_df = df.loc[df.loc[:, "article"] == article].dropna(subset=["affil_name"])
    article_df = article_df.sort_values("citations", ascending=False)
    article_df = drop_and_count_duplicates(
        article_df, ["affil_name"], col_name="cited_by_affil"
    )
    display(article_df.loc[:, cols].head(5))

Pretraining Codomain Attention Neural Operators for Solving Multiphysics PDEs


,citations,affil_name,affil_country_name,cited_by_affil
6,27728.0,California Institute of Technology,United States,3
2,2705.0,Frostburg State University,United States,3


DPOT: Auto-Regressive Denoising Operator Transformer for Large-Scale PDE Pre-Training


,citations,affil_name,affil_country_name,cited_by_affil
16,46964.0,University of Pennsylvania,United States,5
11,36042.0,Tsinghua University,China,4
24,9640.0,"Swiss Federal Institute of Technology, Zurich",Switzerland,5


Neural Operators with Localized Integral and Differential Kernels


...

,citations,affil_name,affil_country_name,cited_by_affil
3027,106354.0,Brown University,United States,1
3051,33347.0,University of South Australia,Australia,3
3034,26183.0,Princeton University,United States,5
3041,24534.0,"Swiss Federal Institute of Technology, Zurich",Switzerland,28
2935,24373.0,Universität Graz,Austria,2


Towards a regularity theory for ReLU networks–chain rule and global error estimates


,citations,affil_name,affil_country_name,cited_by_affil
3348,60996.0,University of Florida,United States,7
3378,59082.0,Stanford University,United States,1
3329,10503.0,Westfälische Wilhelms-Universität Münster,Germany,1
3350,8220.0,Humboldt Universität Berlin,Germany,2
3334,3993.0,Universität Vienna,Austria,4


How degenerate is the parametrization of neural networks with the ReLU activation function?


,citations,affil_name,affil_country_name,cited_by_affil
3435,45144.0,University of Manitoba,Canada,1
3446,19980.0,Institute of Science and Technology,Austria,1
3463,19169.0,"Faculty of Engineering, Lund University",Sweden,2
3459,18182.0,Ludwig-Maximilians-Universität München,Germany,3
3458,15776.0,Technische Universität München,Germany,1
